# EJEMPLO DE SEGUIMIENTO DE OBJETOS CON YOLO

## INSTALACIONES PREVIAS

In [1]:
# Modelos de detección
!pip install ultralytics

# Para descargar videos de Google Drive
!pip install -U gdown

# Para representar y procesar vídeos y fotogramas
!pip install -U supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.2 MB/s eta 0:00:00


In [2]:
import supervision as sv
import numpy as np
from ultralytics import YOLO

# Muestra las imagenes y los vídeos en la celda de output
%matplotlib inline

## DESCARGAR EL VIDEO

>Subimos el vídeo a Google Colab y establecemos la ruta al fichero.

In [3]:
SOURCE_VIDEO_PATH = f"trafico.mp4"

## CARGAR EL MODELO PREENTRENADO YOLOv8

In [4]:
MODEL = "yolov8x.pt" # Esta línea asigna el nombre del archivo de pesos preentrenado del modelo a la variable MODEL. El archivo ".pt" es típicamente un archivo de PyTorch que contiene los pesos entrenados del modelo.
model = YOLO(MODEL) # Aquí se crea una instancia del modelo YOLO utilizando el archivo de pesos especificado
model.fuse() #La fusión de capas es una técnica de optimización que combina ciertas capas y operaciones del modelo para reducir la complejidad computacional y mejorar el rendimiento en tiempo de ejecución. Por ejemplo, puede fusionar una capa de convolución seguida de una capa de BatchNorm en una sola capa de convolución

100%|██████████| 131M/131M [00:00<00:00, 301MB/s]


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [5]:
# Diccionario que mapea class_id a class_name
CLASS_NAMES_DICT = model.model.names
CLASS_NAMES_DICT

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [10]:
# class_ids de interés - coche
CLASS_ID = [2]

In [7]:
# Obtener información del video a partir de la ruta del video fuente
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1052, height=498, fps=60, total_frames=309)

>Vemos la resolución del vídeo. La línea de conteo está en la mitad, si la altura total es de 498, la línea está a 249 píxeles.

## PREDECIR Y ANOTAR TODO EL VIDEO

In [8]:
# Definir el punto de inicio de la línea de conteo
# Punto x=50, y=1500
LINE_START = sv.Point(25, 249)

# Definir el punto final de la línea de conteo
# Punto con el mismo margen de pixeles al otro lado.
# El ancho se obtiene del código de la siguiente celda.
LINE_END = sv.Point(1052-25, 249)

# Establecer la ruta del video objetivo donde se guardará el resultado del conteo de personas
TARGET_VIDEO_PATH = f"trafico-result.mp4"

In [ ]:
# Obtener información del video a partir de la ruta del video fuente
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1920, height=1080, fps=30, total_frames=302)

In [22]:
# Crear una instancia de BYTETracker
# Algoritmo de seguimiento de objetos
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)

# Crear una instancia de VideoInfo
# Obtiene información de los metadatos del vídeo
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Crear un generador de fotogramas
# Generará un iterador de fotogramas
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# Crear una instancia de LineCounter (la linea de conteo)
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# Crear instancias de BoxAnnotator, TraceAnnotator y  LineZoneAnnotator
# Cajas delimitadoras
box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=1, text_scale=0.5)
# Trazas (historial de movimiento)
trace_annotator = sv.TraceAnnotator(thickness=0, trace_length=0)
# Línea de conteo.
line_zone_annotator = sv.LineZoneAnnotator(thickness=2, text_thickness=1, text_scale=0.5)

# función a ejecutar después en cada fotograma
# Lleva a cabo la detección, clasificación, seguimiento, conteo...
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # Predicción del modelo para el fotograma y conversión a detecciones de supervisión
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Tener en cuenta solo las clases que queeremos
    detections = detections[np.isin(detections.class_id, CLASS_ID)]

    # seguimiento de las detecciones
    detections = byte_tracker.update_with_detections(detections)

    # Formato de etiquetas
    labels = [
        f"#{tracker_id}"
        for confidence, class_id, tracker_id in zip(
            detections.confidence, detections.class_id, detections.tracker_id
        )
    ]

    # Crea las trazas
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(), detections=detections
    )

    # Pone la caja delimitadora y la etiqueta a las trazas.
    annotated_frame = box_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels
    )

    # update line counter
    # Actualiza el conteo de línea
    line_zone.trigger(detections)

    # return frame with box and line annotated result
    return line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)


# Procesar todo el video
sv.process_video(
    source_path=SOURCE_VIDEO_PATH, target_path=TARGET_VIDEO_PATH, callback=callback
)

>Descarga del vídeo final.

In [23]:
from google.colab import files
files.download(TARGET_VIDEO_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>